In [1]:
# Method 2 below
!pip install torch torchvision sklearn

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to 

In [14]:
# import pandas as pd
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.feature_extraction.text import TfidfVectorizer
# import numpy as np

# # 上传并加载数据
# # uploaded = files.upload()
# df = pd.read_csv('amazon.csv')

# # 数据预处理
# df['actual_price'] = df['actual_price'].str.replace('₹', '').str.replace(',', '').astype(float)

# scaler = StandardScaler()
# df['price_scaled'] = scaler.fit_transform(df[['actual_price']])

# encoder = OneHotEncoder()
# categories_encoded = encoder.fit_transform(df[['category']]).toarray()

# tfidf_vectorizer = TfidfVectorizer(max_features=100)
# names_tfidf = tfidf_vectorizer.fit_transform(df['product_name']).toarray()
# descriptions_tfidf = tfidf_vectorizer.fit_transform(df['about_product']).toarray()

# features = np.hstack([df[['price_scaled']].values, categories_encoded, names_tfidf, descriptions_tfidf])

import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# 上传并加载数据
# uploaded = files.upload()
df = pd.read_csv('amazon_more.csv')

# 数据预处理
# 数据预处理
# 填充 NaN 值为空字符串
df['Selling Price'] = df['Selling Price'].fillna('')

# 去掉 '$' 和 ',' 符号
df['Selling Price'] = df['Selling Price'].str.replace('$', '').str.replace(',', '')

# 对于没有数字的行用 0 代替
df['Selling Price'] = df['Selling Price'].apply(lambda x: '0' if not x.replace('.', '', 1).isdigit() else x)

# 处理有 "-" 的行，取减号前面的值
df['Selling Price'] = df['Selling Price'].apply(lambda x: x.split('-')[0] if '-' in x else x)

# 转换为 float 类型
df['Selling Price'] = df['Selling Price'].astype(float)

####

scaler = StandardScaler()
df['price_scaled'] = scaler.fit_transform(df[['Selling Price']])

encoder = OneHotEncoder()
categories_encoded = encoder.fit_transform(df[['Category']]).toarray()

tfidf_vectorizer = TfidfVectorizer(max_features=100)
names_tfidf = tfidf_vectorizer.fit_transform(df['Product Name']).toarray()
# descriptions_tfidf = tfidf_vectorizer.fit_transform(df['About Product']).toarray()

# 填充 NaN 值为空字符串
df['Product Name'] = df['Product Name'].fillna('')
df['About Product'] = df['About Product'].fillna('')

# 进行 TF-IDF 向量化
tfidf_vectorizer = TfidfVectorizer(max_features=100)
names_tfidf = tfidf_vectorizer.fit_transform(df['Product Name']).toarray()
descriptions_tfidf = tfidf_vectorizer.fit_transform(df['About Product']).toarray()

# 将所有特征合并
features = np.hstack([df[['price_scaled']].values, categories_encoded, names_tfidf, descriptions_tfidf])

# features = np.hstack([df[['price_scaled']].values, categories_encoded, names_tfidf, descriptions_tfidf])

In [16]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset

# class Autoencoder(nn.Module):
#     def __init__(self, input_dim, hidden_dim):
#         super(Autoencoder, self).__init__()
#         self.encoder = nn.Sequential(
#             nn.Linear(input_dim, hidden_dim),
#             nn.ReLU()
#         )
#         self.decoder = nn.Sequential(
#             nn.Linear(hidden_dim, input_dim),
#             nn.ReLU()
#         )

#     def forward(self, x):
#         encoded = self.encoder(x)
#         decoded = self.decoder(encoded)
#         return encoded, decoded

# input_dim = features.shape[1]
# hidden_dim = 50
# autoencoder = Autoencoder(input_dim, hidden_dim)
# criterion = nn.MSELoss()
# optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

# # Convert features to tensor
# features_tensor = torch.FloatTensor(features)
# dataset = TensorDataset(features_tensor, features_tensor)
# dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# # Training the autoencoder
# num_epochs = 100
# for epoch in range(num_epochs):
#     for data in dataloader:
#         inputs, _ = data
#         optimizer.zero_grad()
#         encoded, decoded = autoencoder(inputs)
#         loss = criterion(decoded, inputs)
#         loss.backward()
#         optimizer.step()
#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Method 1, automatically adjust learning rate
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class Autoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim, input_dim),
            nn.ReLU()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

input_dim = features.shape[1]
hidden_dim = 50
autoencoder = Autoencoder(input_dim, hidden_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)

# Convert features to tensor
features_tensor = torch.FloatTensor(features)
dataset = TensorDataset(features_tensor, features_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Training the autoencoder
num_epochs = 100
for epoch in range(num_epochs):
    epoch_loss = 0
    for data in dataloader:
        inputs, _ = data
        optimizer.zero_grad()
        encoded, decoded = autoencoder(inputs)
        loss = criterion(decoded, inputs)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    # 调度学习率
    scheduler.step(epoch_loss)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(dataloader):.4f}')

Epoch [1/100], Loss: 0.0031
Epoch [2/100], Loss: 0.0025
Epoch [3/100], Loss: 0.0023
Epoch [4/100], Loss: 0.0023
Epoch [5/100], Loss: 0.0023
Epoch [6/100], Loss: 0.0023
Epoch [7/100], Loss: 0.0023
Epoch [8/100], Loss: 0.0023
Epoch [9/100], Loss: 0.0023
Epoch [10/100], Loss: 0.0022
Epoch [11/100], Loss: 0.0022
Epoch [12/100], Loss: 0.0022
Epoch [13/100], Loss: 0.0022
Epoch [14/100], Loss: 0.0022
Epoch [15/100], Loss: 0.0022
Epoch [16/100], Loss: 0.0022
Epoch [17/100], Loss: 0.0022
Epoch [18/100], Loss: 0.0022
Epoch [19/100], Loss: 0.0022
Epoch [20/100], Loss: 0.0022
Epoch [21/100], Loss: 0.0022
Epoch [22/100], Loss: 0.0022
Epoch [23/100], Loss: 0.0022
Epoch [24/100], Loss: 0.0022
Epoch [25/100], Loss: 0.0022
Epoch [26/100], Loss: 0.0022
Epoch [27/100], Loss: 0.0022
Epoch [28/100], Loss: 0.0022
Epoch [29/100], Loss: 0.0022
Epoch [30/100], Loss: 0.0022
Epoch [31/100], Loss: 0.0022
Epoch [32/100], Loss: 0.0022
Epoch [33/100], Loss: 0.0022
Epoch [34/100], Loss: 0.0022
Epoch [35/100], Loss: 0

In [17]:
!pip install optuna

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 71.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 614.3/614.3 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.4 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [20]:
!pip install optuna[visualization]

Defaulting to user installation because normal site-packages is not writeable


In [21]:
# Using optuna to finetune hyperparameters
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from optuna.visualization import plot_optimization_history, plot_param_importances

class Autoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim, input_dim),
            nn.ReLU()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

def objective(trial):
    input_dim = features.shape[1]
    hidden_dim = trial.suggest_int('hidden_dim', 10, 100)
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_int('batch_size', 16, 128)

    autoencoder = Autoencoder(input_dim, hidden_dim)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(autoencoder.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)

    features_tensor = torch.FloatTensor(features)
    dataset = TensorDataset(features_tensor, features_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    num_epochs = 100
    best_loss = float('inf')

    for epoch in range(num_epochs):
        epoch_loss = 0
        for data in dataloader:
            inputs, _ = data
            optimizer.zero_grad()
            encoded, decoded = autoencoder(inputs)
            loss = criterion(decoded, inputs)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        epoch_loss /= len(dataloader)
        scheduler.step(epoch_loss)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

        if epoch_loss < best_loss:
            best_loss = epoch_loss

    return best_loss

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

print('Best trial:')
trial = study.best_trial
print(f'  Loss: {trial.value}')
print('  Params:')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

# 可视化优化历史
opt_history = plot_optimization_history(study)
opt_history.show()

# 可视化参数重要性
param_importances = plot_param_importances(study)
param_importances.show()

[I 2024-06-23 06:31:19,956] A new study created in memory with name: no-name-51658c2a-9e4b-446b-a6ec-9d144ebc546e


Epoch [1/100], Loss: 0.0031
Epoch [2/100], Loss: 0.0020
Epoch [3/100], Loss: 0.0019
Epoch [4/100], Loss: 0.0019
Epoch [5/100], Loss: 0.0019
Epoch [6/100], Loss: 0.0019
Epoch [7/100], Loss: 0.0019
Epoch [8/100], Loss: 0.0019
Epoch [9/100], Loss: 0.0019
Epoch [10/100], Loss: 0.0018
Epoch [11/100], Loss: 0.0018
Epoch [12/100], Loss: 0.0018
Epoch [13/100], Loss: 0.0018
Epoch [14/100], Loss: 0.0018
Epoch [15/100], Loss: 0.0018
Epoch [16/100], Loss: 0.0018
Epoch [17/100], Loss: 0.0018
Epoch [18/100], Loss: 0.0018
Epoch [19/100], Loss: 0.0018
Epoch [20/100], Loss: 0.0018
Epoch [21/100], Loss: 0.0018
Epoch [22/100], Loss: 0.0018
Epoch [23/100], Loss: 0.0018
Epoch [24/100], Loss: 0.0018
Epoch [25/100], Loss: 0.0018
Epoch [26/100], Loss: 0.0018
Epoch [27/100], Loss: 0.0018
Epoch [28/100], Loss: 0.0018
Epoch [29/100], Loss: 0.0018
Epoch [30/100], Loss: 0.0018
Epoch [31/100], Loss: 0.0018
Epoch [32/100], Loss: 0.0018
Epoch [33/100], Loss: 0.0018
Epoch [34/100], Loss: 0.0018
Epoch [35/100], Loss: 0

[I 2024-06-23 06:31:40,147] Trial 0 finished with value: 0.0017437033809810168 and parameters: {'hidden_dim': 24, 'lr': 0.004123428867632061, 'batch_size': 96}. Best is trial 0 with value: 0.0017437033809810168.


Epoch [1/100], Loss: 0.0035
Epoch [2/100], Loss: 0.0030
Epoch [3/100], Loss: 0.0031
Epoch [4/100], Loss: 0.0033
Epoch [5/100], Loss: 0.0033
Epoch [6/100], Loss: 0.0033
Epoch [7/100], Loss: 0.0033
Epoch [8/100], Loss: 0.0033
Epoch [9/100], Loss: 0.0033
Epoch [10/100], Loss: 0.0033
Epoch [11/100], Loss: 0.0033
Epoch [12/100], Loss: 0.0033
Epoch [13/100], Loss: 0.0033
Epoch [14/100], Loss: 0.0033
Epoch [15/100], Loss: 0.0033
Epoch [16/100], Loss: 0.0034
Epoch [17/100], Loss: 0.0033
Epoch [18/100], Loss: 0.0033
Epoch [19/100], Loss: 0.0033
Epoch [20/100], Loss: 0.0033
Epoch [21/100], Loss: 0.0033
Epoch [22/100], Loss: 0.0033
Epoch [23/100], Loss: 0.0033
Epoch [24/100], Loss: 0.0033
Epoch [25/100], Loss: 0.0033
Epoch [26/100], Loss: 0.0033
Epoch [27/100], Loss: 0.0033
Epoch [28/100], Loss: 0.0033
Epoch [29/100], Loss: 0.0033
Epoch [30/100], Loss: 0.0033
Epoch [31/100], Loss: 0.0033
Epoch [32/100], Loss: 0.0033
Epoch [33/100], Loss: 0.0033
Epoch [34/100], Loss: 0.0033
Epoch [35/100], Loss: 0

[I 2024-06-23 06:32:02,102] Trial 1 finished with value: 0.0029902648432097866 and parameters: {'hidden_dim': 58, 'lr': 0.08591755996943194, 'batch_size': 89}. Best is trial 0 with value: 0.0017437033809810168.


...

[I 2024-06-23 07:13:13,967] Trial 98 finished with value: 0.0009201584851299244 and parameters: {'hidden_dim': 94, 'lr': 0.0002521022409169737, 'batch_size': 73}. Best is trial 72 with value: 0.0008340583749073135.


Epoch [100/100], Loss: 0.0009
Epoch [1/100], Loss: 0.0034
Epoch [2/100], Loss: 0.0026
Epoch [3/100], Loss: 0.0022
Epoch [4/100], Loss: 0.0018
Epoch [5/100], Loss: 0.0016
Epoch [6/100], Loss: 0.0014
Epoch [7/100], Loss: 0.0013
Epoch [8/100], Loss: 0.0012
Epoch [9/100], Loss: 0.0012
Epoch [10/100], Loss: 0.0011
Epoch [11/100], Loss: 0.0011
Epoch [12/100], Loss: 0.0011
Epoch [13/100], Loss: 0.0011
Epoch [14/100], Loss: 0.0011
Epoch [15/100], Loss: 0.0011
Epoch [16/100], Loss: 0.0011
Epoch [17/100], Loss: 0.0011
Epoch [18/100], Loss: 0.0011
Epoch [19/100], Loss: 0.0011
Epoch [20/100], Loss: 0.0011
Epoch [21/100], Loss: 0.0010
Epoch [22/100], Loss: 0.0010
Epoch [23/100], Loss: 0.0010
Epoch [24/100], Loss: 0.0010
Epoch [25/100], Loss: 0.0010
Epoch [26/100], Loss: 0.0010
Epoch [27/100], Loss: 0.0010
Epoch [28/100], Loss: 0.0010
Epoch [29/100], Loss: 0.0010
Epoch [30/100], Loss: 0.0010
Epoch [31/100], Loss: 0.0010
Epoch [32/100], Loss: 0.0010
Epoch [33/100], Loss: 0.0010
Epoch [34/100], Loss: 

[I 2024-06-23 07:13:41,642] Trial 99 finished with value: 0.0009995704868170261 and parameters: {'hidden_dim': 93, 'lr': 0.00025609553474958876, 'batch_size': 62}. Best is trial 72 with value: 0.0008340583749073135.


Epoch [100/100], Loss: 0.0010
Best trial:
  Loss: 0.0008340583749073135
  Params:
    hidden_dim: 94
    lr: 0.0001343838543590683
    batch_size: 114


ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [23]:
!pip install plotly

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 87.4 MB/s eta 0:00:00:00:0100:01


In [25]:
!pip install plotly optuna transformers datasets accelerate

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.7 MB/s eta 0:00:00
  Using cached torch-2.3.1-cp39-cp39-manylinux1_x86_64.whl.metadata (26 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 97.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [38]:
!ls /home/uc3f57f00092eb0bd6e94396bf95c473/.local/lib/python3.9/site-packages | grep plotly


5092.42s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


_plotly_future_
_plotly_utils
jupyterlab_plotly
plotly
plotly-5.22.0.dist-info


In [39]:
import sys
sys.path.append('/home/uc3f57f00092eb0bd6e94396bf95c473/.local/lib/python3.9/site-packages')

import plotly
print(plotly.__version__)

5.22.0


In [43]:
import optuna
from optuna.visualization import plot_optimization_history
import matplotlib.pyplot as plt

study = optuna.create_study()
study.optimize(lambda trial: trial.suggest_float('x', -10, 10) ** 2, n_trials=100)

plot_optimization_history(study)
plt.show()

[I 2024-06-23 07:29:48,003] A new study created in memory with name: no-name-e04b6c02-2820-409d-8116-18ebd7bca0f1
[I 2024-06-23 07:29:48,009] Trial 0 finished with value: 4.910593706909518 and parameters: {'x': 2.215985944655227}. Best is trial 0 with value: 4.910593706909518.
[I 2024-06-23 07:29:48,013] Trial 1 finished with value: 65.94210737548701 and parameters: {'x': 8.120474578218136}. Best is trial 0 with value: 4.910593706909518.
[I 2024-06-23 07:29:48,017] Trial 2 finished with value: 59.00764473929031 and parameters: {'x': 7.681643361891407}. Best is trial 0 with value: 4.910593706909518.
[I 2024-06-23 07:29:48,021] Trial 3 finished with value: 30.703608306522202 and parameters: {'x': 5.5410836761884585}. Best is trial 0 with value: 4.910593706909518.
[I 2024-06-23 07:29:48,024] Trial 4 finished with value: 1.1042065732929105 and parameters: {'x': -1.0508123397129054}. Best is trial 4 with value: 1.1042065732929105.
[I 2024-06-23 07:29:48,028] Trial 5 finished with value: 81.

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [26]:
from optuna.visualization import plot_intermediate_values, plot_parallel_coordinate, plot_slice, plot_contour

# 可视化中间值
intermediate_values = plot_intermediate_values(study)
intermediate_values.show()

# 可视化并行坐标
parallel_coordinate = plot_parallel_coordinate(study)
parallel_coordinate.show()

# 可视化切片图
slice_plot = plot_slice(study)
slice_plot.show()

# 可视化等高线图
contour_plot = plot_contour(study)
contour_plot.show()

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [44]:
# 获取低维嵌入
with torch.no_grad():
    embeddings = autoencoder.encoder(features_tensor)

embeddings = embeddings.numpy()

# 计算余弦相似度
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(embeddings)

# 确保相似度在0到1之间
similarity_matrix = (similarity_matrix - similarity_matrix.min()) / (similarity_matrix.max() - similarity_matrix.min())

# 保存相似度矩阵到CSV文件
similarity_df = pd.DataFrame(similarity_matrix, index=df['Uniq Id'], columns=df['Uniq Id'])
similarity_df.to_csv('product_similarity_more.csv')


In [45]:
def get_recommendations(product_id, top_k=10):
    product_index = df.index[df['Uniq Id'] == product_id][0]
    sim_scores = similarity_matrix[product_index]
    top_indices = np.argsort(sim_scores)[-top_k-1:-1][::-1]
    return df.iloc[top_indices][['Uniq Id', 'Product Name', 'Category', 'Selling Price']]

# 示例：获取商品ID为某个商品的推荐列表
recommended_products = get_recommendations(df['Uniq Id'].iloc[0])
recommended_products

,Uniq Id,Product Name,Category,Selling Price
9695,18726039d068d9422c00da30b8275d3c,DB Longboards Contra Drop Deck Maple Longboard...,Sports & Outdoors | Outdoor Recreation | Skate...,80.96
9064,f098f7ce1cb9c2c6168eaa7499229518,"DB Longboards Phase 38"" Maple Drop Through Lon...",Sports & Outdoors | Outdoor Recreation | Skate...,134.82
122,f880bf9d197e7e4fc752ca2d108cd67c,Enchantimals Styling Head- Brown Mailer,Toys & Games | Novelty & Gag Toys,18.59
8952,c9d8840a3e70200ccdf51ae87223bc2d,Barbie Dreamhouse [Amazon Exclusive],Toys & Games | Dolls & Accessories | Dollhouses,199.99
4771,08cc4c151e98aa6b2deb09c3a189f4ee,Power Rangers Movie Legacy Coins,Toys & Games | Dress Up & Pretend Play | Prete...,51.92
5425,9d1c1a976967938979902d35c552321d,Bigjigs Rail BJT415 Bulldozer Low Loader,Toys & Games | Play Vehicles | Play Trains & R...,15.00
2069,95869e007e113c281b757ef88808ffe8,T.O.T.S. Nursery Stroller Packaging,Toys & Games | Dress Up & Pretend Play | Prete...,39.97
7243,c7d7e6bb3af8ef90a215300b7eda296d,PLAYMOBIL Porsche Macan GTS,Toys & Games | Toy Figures & Playsets | Playse...,56.93
6229,761324a7ba9747ec092dfdffbc8292e3,Watchover Voodoo Aka-Oni Voodoo Novelty,Toys & Games | Dolls & Accessories | Dolls,11.02
4617,70e2b1e67deae362f45a9ca86d6c1882,Sleeves: Dragon Shield Matte Japanese Silver (60),Toys & Games | Collectible Toys | Collectible ...,7.95
